In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [2]:
import pandas as pd
from bayesian.train_bn import structure_learning
from preprocess.discretization import get_nodes_type, discretization, code_categories
from scipy import stats
import math
import random
from graph.precision_recall import precision_recall
import time

In [3]:
method = 'LL'
steps = 5

In [4]:
true_net = [['Region', 'wd'],
 ['Region', 'blh'],
 ['Region', 'CVD60'],
 ['Region', 'pm2.5'],
 ['Zone', 'no2'],
 ['Zone', 'o3'],
 ['Zone', 'so2'],
 ['Zone', 'co'],
 ['Zone', 'pm10'],
 ['Type', 'no2'],
 ['Type', 'o3'],
 ['Type', 'so2'],
 ['Type', 'co'],
 ['Type', 'pm10'],
 ['Type', 'pm2.5'],
 ['Year', 't2m'],
 ['Year', 'ws'],
 ['Year', 'wd'],
 ['Year', 'tp'],
 ['Year', 'CVD60'],
 ['Year', 'no2'],
 ['Year', 'o3'],
 ['Year', 'so2'],
 ['Year', 'co'],
 ['Year', 'pm10'],
 ['Year', 'pm2.5'],
 ['Season', 'CVD60'],
 ['Month', 't2m'],
 ['Month', 'ws'],
 ['Month', 'wd'],
 ['Month', 'tp'],
 ['Month', 'blh'],
 ['Month', 'CVD60'],
 ['Day', 'ws'],
 ['Day', 'wd'],
 ['Hour', 't2m'],
 ['Hour', 'tp'],
 ['Hour', 'blh'],
 ['Hour', 'ssr'],
 ['Latitude', 't2m'],
 ['Latitude', 'ws'],
 ['Latitude', 'ssr'],
 ['Longitude', 'ws'],
 ['Longitude', 'blh'],
 ['Longitude', 'ssr'],
 ['Altitude', 'ssr'],
 ['Altitude', 'no2'],
 ['t2m', 'ws'],
 ['t2m', 'blh'],
 ['t2m', 'ssr'],
 ['ws', 'tp'],
 ['ws', 'blh'],
 ['ws', 'ssr'],
 ['wd', 't2m'],
 ['wd', 'ws'],
 ['wd', 'blh'],
 ['wd', 'ssr'],
 ['blh', 'ssr'],
 ['blh', 'no2'],
 ['blh', 'o3'],
 ['ssr', 'tp'],
 ['no2', 'ssr'],
 ['o3', 'ssr'],
 ['o3', 'no2'],
 ['so2', 'ssr'],
 ['co', 'ssr'],
 ['co', 'no2'],
 ['co', 'so2'],
 ['pm10', 'ssr'],
 ['pm10', 'no2'],
 ['pm10', 'pm2.5']]

In [5]:
df_result = pd.DataFrame(columns = ['Strategy', 'Alg', 'AP', 'AR', 'AHP', 'AHR', 'SHD', 'Time'])

In [6]:
geo = pd.read_csv('../data/mehra.csv')
columns = ['Region', 'Zone', 'Type', 'Year', 'Season', 'Month', 'Day', 'Hour',
       'Latitude', 'Longitude', 'Altitude', 't2m', 'ws', 'wd', 'tp', 'blh',
       'ssr', 'CVD60', 'no2', 'o3', 'so2', 'co', 'pm10', 'pm2.5']
geo = geo[columns]
geo.dropna(inplace=True)
geo.reset_index(inplace=True, drop=True)
geo

,Region,Zone,Type,Year,Season,Month,Day,Hour,Latitude,Longitude,...,tp,blh,ssr,CVD60,no2,o3,so2,co,pm10,pm2.5
0,Yorkshire and The Humber,West Midlands Urban Area,Background Urban,1988,Winter,3,26,9,52.829361,-1.726818,...,0.000784,992.501830,5.133717e+05,0.205020,61.974631,17.740408,13.813847,1.442624,36.486336,18.646589
1,North West,Greater London Urban Area,Background Rural,1995,Spring,8,2,19,51.377194,-2.745956,...,0.000130,1086.774403,1.149178e+07,0.246176,83.165539,32.657576,22.427120,1.273507,34.964375,18.643421
2,West Midlands,North West & Merseyside,Background Urban,1997,Spring,3,14,4,52.002869,-2.760588,...,0.000209,683.612000,-1.076695e+01,0.181319,76.460115,17.692036,13.803708,1.442563,36.458572,18.645740
3,South East,Greater London Urban Area,Traffic Urban,1994,Winter,3,3,18,54.061788,-2.439756,...,0.000996,1252.022401,3.290441e+06,0.201057,85.786023,17.749945,26.277995,1.793299,36.543445,18.641970
4,South East,Greater London Urban Area,Background Urban,1997,Fall,8,15,18,54.570412,-1.460727,...,-0.000691,1730.349463,8.348839e+06,0.175219,48.424476,83.015267,-0.018112,0.359330,38.317914,18.640665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Yorkshire and The Humber,North West & Merseyside,Background Urban,1993,Spring,7,3,0,51.502964,0.300996,...,0.003594,193.882528,4.162147e+06,0.175375,76.300697,17.760820,13.802362,1.442467,36.448082,18.639546
9996,Greater London Authority,Eastern,Traffic Urban,1997,Winter,4,27,3,52.740712,-0.547268,...,0.001259,28.284668,-2.723962e+05,0.282495,-97.144486,17.749629,13.818443,1.385102,36.086071,18.642884
9997,East Midlands,Greater London Urban Area,Traffic Urban,1985,Summer,7,4,23,53.554519,-1.734243,...,0.002055,407.258881,1.815961e+06,0.134340,78.544941,17.735576,13.808181,1.442177,36.476172,18.645422
9998,Yorkshire and The Humber,West Midlands,Background Urban,1990,Fall,10,20,20,51.180594,0.098250,...,0.000620,35.156433,1.742372e+06,0.219353,76.320929,17.742210,13.792902,1.442187,36.480802,18.642345


In [7]:
geo['Year'] = geo['Year'].apply(lambda x: str(x))
geo['Month'] = geo['Month'].apply(lambda x: str(x))
geo['Day'] = geo['Day'].apply(lambda x: str(x))
geo['Hour'] = geo['Hour'].apply(lambda x: str(x))

In [8]:
geo_types = get_nodes_type(geo)
geo_types

{'Region': 'disc',
 'Zone': 'disc',
 'Type': 'disc',
 'Year': 'disc',
 'Season': 'disc',
 'Month': 'disc',
 'Day': 'disc',
 'Hour': 'disc',
 'Latitude': 'cont',
 'Longitude': 'cont',
 'Altitude': 'cont',
 't2m': 'cont',
 'ws': 'cont',
 'wd': 'cont',
 'tp': 'cont',
 'blh': 'cont',
 'ssr': 'cont',
 'CVD60': 'cont',
 'no2': 'cont',
 'o3': 'cont',
 'so2': 'cont',
 'co': 'cont',
 'pm10': 'cont',
 'pm2.5': 'cont'}

In [9]:
colums_for_code = []
columns_for_disc = []
for c in columns:
    if geo_types[c] == 'disc':
        colums_for_code.append(c)
    else:
        columns_for_disc.append(c)

In [10]:
geo_coded, label_coder = code_categories(geo, 'label', colums_for_code)
geo_discrete, coder = discretization(geo_coded, 'equal_frequency', columns_for_disc)

# Hill Climbing

# Дискретная структра

In [11]:
for _ in range(steps):
    random.shuffle(columns)
    geo_coded = geo_coded[columns]
    geo_discrete = geo_discrete[columns]
    start_time = time.time()
    bn_geo = structure_learning(geo_discrete, 'HC', geo_types, method)
    full_time = time.time() - start_time
    result = precision_recall(bn_geo['E'], true_net, 3)
    df_result = df_result.append({'Strategy': 'HC', 'Alg':'D', **result, 'Time': full_time}, ignore_index=True)

# Смешанная структура

In [12]:
for _ in range(steps):
    random.shuffle(columns)
    geo_coded = geo_coded[columns]
    geo_discrete = geo_discrete[columns]
    start_time = time.time()
    bn_geo = structure_learning(geo_coded, 'HC', geo_types, method)
    full_time = time.time() - start_time
    result = precision_recall(bn_geo['E'], true_net, 3)
    df_result = df_result.append({'Strategy': 'HC', 'Alg':'M', **result, 'Time': full_time}, ignore_index=True)

# EVO

# Дискретная стурктура

In [13]:
for _ in range(steps):
    random.shuffle(columns)
    geo_coded = geo_coded[columns]
    geo_discrete = geo_discrete[columns]
    start_time = time.time()
    bn_geo = structure_learning(geo_discrete, 'evo', geo_types, method)
    full_time = time.time() - start_time
    result = precision_recall(bn_geo['E'], true_net, 3)
    df_result = df_result.append({'Strategy': 'evo', 'Alg':'D', **result, 'Time': full_time}, ignore_index=True)

Best metric is -25848.0
Generation num: 0
max_depth: 8, no improvements: 0
spent time: 1.1 min
Best metric is -25858.0
Generation num: 1
max_depth: 8, no improvements: 0
spent time: 2.0 min
Best metric is -26029.0
Generation num: 2
max_depth: 8, no improvements: 0
spent time: 2.7 min
Best metric is -26049.0
Generation num: 3
max_depth: 8, no improvements: 0
spent time: 3.5 min
Best metric is -26127.000000000007
Generation num: 4
max_depth: 8, no improvements: 0
spent time: 4.5 min
Best metric is -57464.0
Generation num: 5
max_depth: 8, no improvements: 0
spent time: 5.3 min
Best metric is -58177.0
Generation num: 6
max_depth: 8, no improvements: 0
spent time: 6.3 min
Best metric is -58335.0
Generation num: 7
max_depth: 8, no improvements: 0
spent time: 7.4 min
Best metric is -58345.000000000015
Generation num: 8
max_depth: 8, no improvements: 0
spent time: 8.5 min
Best metric is -79427.0
Generation num: 9
max_depth: 8, no improvements: 0
spent time: 9.7 min
Best metric is -79442.0
Gene

# Смешанная структура

In [14]:
for _ in range(steps):
    random.shuffle(columns)
    geo_coded = geo_coded[columns]
    geo_discrete = geo_discrete[columns]
    start_time = time.time()
    bn_geo = structure_learning(geo_discrete, 'evo', geo_types, method)
    full_time = time.time() - start_time
    result = precision_recall(bn_geo['E'], true_net, 3)
    df_result = df_result.append({'Strategy': 'evo', 'Alg':'M', **result, 'Time': full_time}, ignore_index=True)

Best metric is -38757.0
Generation num: 0
max_depth: 8, no improvements: 0
Number of mutation attempts exceeded. Please check composer requirements for correctness.
spent time: 1.8 min
Best metric is -38757.0
Generation num: 1
max_depth: 8, no improvements: 1
spent time: 2.3 min
Best metric is -39425.0
Generation num: 2
max_depth: 8, no improvements: 0
spent time: 3.0 min
Best metric is -39698.0
Generation num: 3
max_depth: 8, no improvements: 0
spent time: 3.6 min
Best metric is -39805.0
Generation num: 4
max_depth: 8, no improvements: 0
spent time: 4.4 min
Best metric is -39904.0
Generation num: 5
max_depth: 8, no improvements: 0
spent time: 5.0 min
Best metric is -54816.0
Generation num: 6
max_depth: 8, no improvements: 0
spent time: 5.7 min
Best metric is -89050.0
Generation num: 7
max_depth: 8, no improvements: 0
spent time: 6.4 min
Best metric is -90215.0
Generation num: 8
max_depth: 8, no improvements: 0
spent time: 7.3 min
Best metric is -90251.0
Generation num: 9
max_depth: 8,

In [15]:
df_result

,Strategy,Alg,AP,AR,AHP,AHR,SHD,Time
0,HC,D,0.026,0.014,0.026,0.014,108,90.041033
1,HC,D,0.025,0.014,0.025,0.014,109,95.131748
2,HC,D,0.026,0.014,0.026,0.014,108,103.404758
3,HC,D,0.000,0.000,0.000,0.000,110,95.698865
4,HC,D,0.000,0.000,0.000,0.000,109,100.757453
5,HC,M,0.051,0.028,0.026,0.014,107,455.940761
6,HC,M,0.029,0.014,0.000,0.000,104,460.058681
7,HC,M,0.029,0.014,0.029,0.014,103,426.114646
8,HC,M,0.029,0.014,0.029,0.014,103,420.443287
9,HC,M,0.029,0.014,0.000,0.000,104,417.478684


In [16]:
df_result.to_csv('mehra_results_graph.csv', index=False)